https://github.com/vanvalenlab/deepcell-tf/blob/185406798a76cf52a812c650303200fa84af6d9d/scripts/misc/Model_Metrics.ipynb

In [24]:
import os
import sys

# sys.path.insert(0,'/home/sunnycui/deepcell-tf/') 
print(sys.path)

import numpy as np

import matplotlib.pyplot as plt
from skimage.io import imshow

from skimage.measure import label

import keras

from deepcell import metrics

['/home/sunnycui/deepcell-tf', '/home/sunnycui/deepcell-tf', '/home/sunnycui/deepcell-tf', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '', '/home/sunnycui/.local/lib/python3.5/site-packages', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/home/sunnycui/.local/lib/python3.5/site-packages/IPython/extensions', '/home/sunnycui/.ipython']


In [25]:
from deepcell.utils.data_utils import get_data

data_filename = '3T3_NIH.npz'
train_dict, test_dict = get_data(data_filename, test_size=0.2)
print('X.shape: {}\ny.shape: {}'.format(train_dict['X'].shape, train_dict['y'].shape))

X.shape: (230, 30, 154, 182, 1)
y.shape: (230, 30, 154, 182, 1)


In [3]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import callbacks
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.layers import MaxPool3D, Conv3DTranspose, UpSampling3D
from scripts.recurr_gru.conv_gru_layer import ConvGRU2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout, LeakyReLU
from tensorflow.python.keras.layers import Conv3D, ZeroPadding3D, ConvLSTM2D, Cropping3D
from tensorflow.python.keras.layers import Input, Add, Concatenate, Flatten
from tensorflow.python.keras.engine.input_layer import InputLayer

from tensorflow.python.keras.models import Model

from tensorflow.python.keras.regularizers import l2
from deepcell.layers import ImageNormalization3D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Softmax

from deepcell.layers import TensorProduct, ReflectionPadding3D, DilatedMaxPool3D


In [34]:
def feature_net_3D(receptive_field=61,
                      n_frames=3,
                      input_shape=(5, 256, 256, 1),
                      n_features=3,
                      n_channels=1,
                      reg=1e-5,
                      n_conv_filters=64,
                      n_dense_filters=200,
                      VGG_mode=False,
                      init='he_normal',
                      norm_method='std',
                    gru=False,
                      location=False,
                      dilated=False,
                      padding=False,
                      padding_mode='reflect',
                      multires=False,
                      include_top=True):
    # Create layers list (x) to store all of the layers.
    # We need to use the functional API to enable the multiresolution mode
    x = []

    win = (receptive_field - 1) // 2
    win_z = (n_frames - 1) // 2

    if dilated:
        padding = True

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
        time_axis = 2
        row_axis = 3
        col_axis = 4
        if not dilated:
            input_shape = (n_channels, n_frames, receptive_field, receptive_field)
    else:
        channel_axis = -1
        time_axis = 1
        row_axis = 2
        col_axis = 3
        if not dilated:
            input_shape = (n_frames, receptive_field, receptive_field, n_channels)

    x.append(Input(shape=input_shape))
    # x.append(ImageNormalization3D(norm_method=norm_method, filter_size=receptive_field)(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))

    if padding:
        if padding_mode == 'reflect':
            x.append(ReflectionPadding3D(padding=(win_z, win, win))(x[-1]))
        elif padding_mode == 'zero':
            x.append(ZeroPadding3D(padding=(win_z, win, win))([-1]))

    if location:
        x.append(Location3D(in_shape=tuple(x[-1].shape.as_list()[1:]))(x[-1]))
        x.append(Concatenate(axis=channel_axis)([x[-2], x[-1]]))

    if multires:
        layers_to_concat = []

    rf_counter = receptive_field
    block_counter = 0
    d = 1

    while rf_counter > 4:
        filter_size = 3 if rf_counter % 2 == 0 else 4
        x.append(Conv3D(n_conv_filters, (1, filter_size, filter_size), dilation_rate=(1, d, d), kernel_initializer=init, padding='valid', kernel_regularizer=l2(reg))(x[-1]))
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        x.append(Activation('relu')(x[-1]))

        block_counter += 1
        rf_counter -= filter_size - 1

        if block_counter % 2 == 0:
            if dilated:
                x.append(DilatedMaxPool3D(dilation_rate=(1, d, d), pool_size=(1, 2, 2))(x[-1]))
                d *= 2
            else:
                x.append(MaxPool3D(pool_size=(1, 2, 2))(x[-1]))

            if VGG_mode:
                n_conv_filters *= 2

            rf_counter = rf_counter // 2

            if multires:
                layers_to_concat.append(len(x) - 1)

    if multires:
        c = []
        for l in layers_to_concat:
            output_shape = x[l].get_shape().as_list()
            target_shape = x[-1].get_shape().as_list()
            time_crop = (0, 0)

            row_crop = int(output_shape[row_axis] - target_shape[row_axis])

            if row_crop % 2 == 0:
                row_crop = (row_crop // 2, row_crop // 2)
            else:
                row_crop = (row_crop // 2, row_crop // 2 + 1)

            col_crop = int(output_shape[col_axis] - target_shape[col_axis])

            if col_crop % 2 == 0:
                col_crop = (col_crop // 2, col_crop // 2)
            else:
                col_crop = (col_crop // 2, col_crop // 2 + 1)

            cropping = (time_crop, row_crop, col_crop)

            c.append(Cropping3D(cropping=cropping)(x[l]))
        x.append(Concatenate(axis=channel_axis)(c))

    x.append(Conv3D(n_dense_filters, (1, rf_counter, rf_counter), dilation_rate=(1, d, d), kernel_initializer=init, padding='valid', kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))

    x.append(Conv3D(n_dense_filters, (n_frames, 1, 1), dilation_rate=(1, d, d), kernel_initializer=init, padding='valid', kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))

    x.append(TensorProduct(n_dense_filters, kernel_initializer=init, kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))
    
    if gru == True:
        x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(3, 3),
                            activation = 'relu', 
                            padding='same', kernel_initializer=init,
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(3, 3),
                            activation = 'relu', 
                            padding='same', kernel_initializer=init,
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))


    x.append(TensorProduct(n_features, kernel_initializer=init, kernel_regularizer=l2(reg))(x[-1]))

    if not dilated:
        x.append(Flatten()(x[-1]))

    if include_top:
        x.append(Softmax(axis=channel_axis)(x[-1]))

    model = Model(inputs=x[0], outputs=x[-1])
    model.summary()

    return model

In [35]:

def feature_net_skip_3D(receptive_field=61,
                           input_shape=(5, 256, 256, 1),
                           fgbg_model=None,
                        gru=False,
                           last_only=True,
                           n_skips=2,
                           norm_method='std',
                           padding_mode='reflect',
                           **kwargs):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    inputs = Input(shape=input_shape)
    # img = ImageNormalization3D(norm_method=norm_method, filter_size=receptive_field)(inputs)
    img = BatchNormalization(axis=channel_axis)(inputs)

    models = []
    model_outputs = []

    if fgbg_model is not None:
        for layer in fgbg_model.layers:
            layer.trainable = False
        models.append(fgbg_model)
        fgbg_output = fgbg_model(inputs)
        if isinstance(fgbg_output, list):
            fgbg_output = fgbg_output[-1]
        model_outputs.append(fgbg_output)

    for _ in range(n_skips + 1):
        if model_outputs:
            model_input = Concatenate(axis=channel_axis)([img, model_outputs[-1]])
        else:
            model_input = img
        new_input_shape = model_input.get_shape().as_list()[1:]
        models.append(feature_net_3D(receptive_field=receptive_field, 
                                     input_shape=new_input_shape, norm_method=None, dilated=True, 
                                     padding=True, padding_mode=padding_mode, gru=gru, **kwargs))
        model_outputs.append(models[-1](model_input))

    if last_only:
        model = Model(inputs=inputs, outputs=model_outputs[-1])
    else:
        if fgbg_model is None:
            model = Model(inputs=inputs, outputs=model_outputs)
        else:
            model = Model(inputs=inputs, outputs=model_outputs[1:])

    return model

In [73]:
fgbg_gru_model = feature_net_skip_3D(
        receptive_field=61,
        n_features=2,
        n_frames=3,
        n_skips=1,
        gru=True,
        n_conv_filters=32,
        n_dense_filters=128,
        input_shape=tuple(train_dict['X'].shape[1:]),
        multires=False,
        last_only=False,
        norm_method='std')

conv_gru_model = feature_net_skip_3D(
        # fgbg_model=run_fgbg_model,
        receptive_field=61,
        n_skips=1,
        n_features=4,  # (background edge, interior edge, cell interior, background)
        n_frames=3,
        n_conv_filters=32,
        n_dense_filters=128,
        gru=True,
        multires=False,
        last_only=False,
        input_shape=tuple(train_dict['X'].shape[1:]),
        norm_method='std')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_45 (InputLayer)        (None, 30, 154, 182, 1)   0         
_________________________________________________________________
batch_normalization_320 (Bat (None, 30, 154, 182, 1)   4         
_________________________________________________________________
reflection_padding3d_28 (Ref (None, 32, 214, 242, 1)   0         
_________________________________________________________________
conv3d_224 (Conv3D)          (None, 32, 211, 239, 32)  544       
_________________________________________________________________
batch_normalization_321 (Bat (None, 32, 211, 239, 32)  128       
_________________________________________________________________
activation_252 (Activation)  (None, 32, 211, 239, 32)  0         
_________________________________________________________________
conv3d_225 (Conv3D)          (None, 32, 209, 237, 32)  9248      
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_48 (InputLayer)        (None, 30, 154, 182, 1)   0         
_________________________________________________________________
batch_normalization_345 (Bat (None, 30, 154, 182, 1)   4         
_________________________________________________________________
reflection_padding3d_30 (Ref (None, 32, 214, 242, 1)   0         
_________________________________________________________________
conv3d_240 (Conv3D)          (None, 32, 211, 239, 32)  544       
_________________________________________________________________
batch_normalization_346 (Bat (None, 32, 211, 239, 32)  128       
_________________________________________________________________
activation_270 (Activation)  (None, 32, 211, 239, 32)  0         
_________________________________________________________________
conv3d_241 (Conv3D)          (None, 32, 209, 237, 32)  9248      
__________

In [72]:
fgbg_model = feature_net_skip_3D(
        receptive_field=61,
        n_features=2,
        n_frames=3,
        n_skips=1,
        n_conv_filters=32,
        n_dense_filters=128,
        gru=False,
        input_shape=tuple(train_dict['X'].shape[1:]),
        multires=False,
        last_only=False,
        norm_method='std')

conv_model = feature_net_skip_3D(
        # fgbg_model=run_fgbg_model,
        receptive_field=61,
        n_skips=1,
        n_features=4,  # (background edge, interior edge, cell interior, background)
        n_frames=3,
        n_conv_filters=32,
        n_dense_filters=128,
        gru=False,
        multires=False,
        last_only=False,
        input_shape=tuple(train_dict['X'].shape[1:]),
        norm_method='std')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        (None, 30, 154, 182, 1)   0         
_________________________________________________________________
batch_normalization_278 (Bat (None, 30, 154, 182, 1)   4         
_________________________________________________________________
reflection_padding3d_24 (Ref (None, 32, 214, 242, 1)   0         
_________________________________________________________________
conv3d_192 (Conv3D)          (None, 32, 211, 239, 32)  544       
_________________________________________________________________
batch_normalization_279 (Bat (None, 32, 211, 239, 32)  128       
_________________________________________________________________
activation_216 (Activation)  (None, 32, 211, 239, 32)  0         
_________________________________________________________________
conv3d_193 (Conv3D)          (None, 32, 209, 237, 32)  9248      
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_42 (InputLayer)        (None, 30, 154, 182, 1)   0         
_________________________________________________________________
batch_normalization_299 (Bat (None, 30, 154, 182, 1)   4         
_________________________________________________________________
reflection_padding3d_26 (Ref (None, 32, 214, 242, 1)   0         
_________________________________________________________________
conv3d_208 (Conv3D)          (None, 32, 211, 239, 32)  544       
_________________________________________________________________
batch_normalization_300 (Bat (None, 32, 211, 239, 32)  128       
_________________________________________________________________
activation_234 (Activation)  (None, 32, 211, 239, 32)  0         
_________________________________________________________________
conv3d_209 (Conv3D)          (None, 32, 209, 237, 32)  9248      
__________

In [40]:
MODEL_DIR = os.path.join(sys.path[0], 'scripts/recurr_gru/models')

fgbg_gru_model_name = 'fgbg_gru_featurenet_model'
fgbg_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_gru_model_name))

conv_gru_model_name = 'conv_gru_featurenet_model'
conv_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(conv_gru_model_name))

fgbg_model_name = 'fgbg_featurenet_model'
fgbg_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_model_name))

conv_model_name = 'conv_featurenet_model'
conv_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(conv_model_name))

# FGBG Model

In [41]:
fgbg_gru_model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),metrics=['accuracy'])
fgbg_gru_model.load_weights(fgbg_gru_weights_file)

fgbg_model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),metrics=['accuracy'])
fgbg_model.load_weights(fgbg_weights_file)

In [42]:
X_test, y_test = test_dict['X'][:4], test_dict['y'][:4]
predict_gru = fgbg_gru_model.predict(X_test)[-1]

X_test, y_test = test_dict['X'][:4], test_dict['y'][:4]
predict = fgbg_model.predict(X_test)[-1]

In [43]:
print(predict.shape, predict_gru.shape)

(4, 30, 154, 182, 2) (4, 30, 154, 182, 2)


In [58]:
i = np.random.randint(low=0, high=predict.shape[0])
frame = np.random.randint(low=0, high=predict.shape[1])
print(i, frame)

predict_lbl_gru = label((predict_gru[:,frame,:,:,1]>0.5).astype('int'))
predict_lbl = label((predict[:,frame,:,:,1]>0.5).astype('int'))

3 0


### Featurenet (no GRU)

In [59]:
y_true = label(y_test[i:i+1,frame,:,:,0].astype('int'))
y_pred = label(predict_lbl[i:i+1])

fig,ax = plt.subplots(1,3,figsize=(10,8))
ax[0].imshow(predict[i, frame, :,:,1])
ax[0].set_title('Prediction')
# Repeat labeling to get number assignments in range for this particular frame
ax[1].imshow(y_pred[0],cmap='jet')
ax[1].set_title('Labeled Prediction')
ax[2].imshow(y_true[0],cmap='jet')
ax[2].set_title('Labeled Truth')

m = metrics.Metrics('singleton',seg=True)
m.calc_object_stats(y_true,y_pred)

INFO:tensorflow:0 samples processed

____________Object-based statistics____________

Number of true cells:		 5
Number of predicted cells:	 5

True positives:  5	Accuracy:   100.0%

False positives: 0	Perc Error: nan%
False negatives: 0	Perc Error: nan%
Merges:		 0	Perc Error: nan%
Splits:		 0	Perc Error: nan%

SEG: 0.8311 



/home/sunnycui/deepcell-tf/deepcell/metrics.py:635: RuntimeWarning: invalid value encountered in long_scalars
  100 * round(self.stats['false_pos'].sum() / total_err, 4)))
/home/sunnycui/deepcell-tf/deepcell/metrics.py:638: RuntimeWarning: invalid value encountered in long_scalars
  100 * round(self.stats['false_neg'].sum() / total_err, 4)))
/home/sunnycui/deepcell-tf/deepcell/metrics.py:641: RuntimeWarning: invalid value encountered in long_scalars
  100 * round(self.stats['merge'].sum() / total_err, 4)))
/home/sunnycui/deepcell-tf/deepcell/metrics.py:644: RuntimeWarning: invalid value encountered in long_scalars
  100 * round(self.stats['split'].sum() / total_err, 4)))


### Featurenet with GRU

In [60]:
y_true = label(y_test[i:i+1,frame,:,:,0].astype('int'))
y_pred = label(predict_lbl_gru[i:i+1])

fig,ax = plt.subplots(1,3,figsize=(10,8))
ax[0].imshow(predict_gru[i, frame, :,:,1])
ax[0].set_title('Prediction')
# Repeat labeling to get number assignments in range for this particular frame
ax[1].imshow(y_pred[0],cmap='jet')
ax[1].set_title('Labeled Prediction')
ax[2].imshow(y_true[0],cmap='jet')
ax[2].set_title('Labeled Truth')

m = metrics.Metrics('singleton',seg=True)
m.calc_object_stats(y_true,y_pred)

INFO:tensorflow:0 samples processed

____________Object-based statistics____________

Number of true cells:		 5
Number of predicted cells:	 5

True positives:  5	Accuracy:   100.0%

False positives: 0	Perc Error: nan%
False negatives: 0	Perc Error: nan%
Merges:		 0	Perc Error: nan%
Splits:		 0	Perc Error: nan%

SEG: 0.8885 



# Run metrics on complete test set

In [61]:
from importlib import reload

reload(metrics)

<module 'deepcell.metrics' from '/home/sunnycui/deepcell-tf/deepcell/metrics.py'>

### Featurenet (no GRU)

In [62]:
%%time
m = metrics.Metrics('fgbg',seg=True)

y_true_lbl = y_test[:,frame,:,:,0].astype('int')
y_pred_lbl = predict_lbl

m.calc_object_stats(y_true_lbl,y_pred_lbl)

INFO:tensorflow:0 samples processed

____________Object-based statistics____________

Number of true cells:		 32
Number of predicted cells:	 32

True positives:  32	Accuracy:   100.0%

False positives: 0	Perc Error: nan%
False negatives: 0	Perc Error: nan%
Merges:		 0	Perc Error: nan%
Splits:		 0	Perc Error: nan%

SEG: 0.824 

CPU times: user 110 ms, sys: 10.4 ms, total: 120 ms
Wall time: 118 ms


### Featurenet with GRU

In [63]:
%%time
m = metrics.Metrics('fgbg',seg=True)

y_true_gru_lbl = y_test[:,frame,:,:,0].astype('int')
y_pred_gru_lbl = predict_lbl_gru

m.calc_object_stats(y_true_gru_lbl,y_pred_gru_lbl)

INFO:tensorflow:0 samples processed

____________Object-based statistics____________

Number of true cells:		 32
Number of predicted cells:	 32

True positives:  32	Accuracy:   100.0%

False positives: 0	Perc Error: nan%
False negatives: 0	Perc Error: nan%
Merges:		 0	Perc Error: nan%
Splits:		 0	Perc Error: nan%

SEG: 0.8342 

CPU times: user 117 ms, sys: 755 µs, total: 118 ms
Wall time: 116 ms


# Convolution model

In [74]:
conv_gru_model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),metrics=['accuracy'])
conv_gru_model.load_weights(conv_gru_weights_file)

conv_model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),metrics=['accuracy'])
conv_model.load_weights(conv_weights_file)

In [75]:
X_test, y_test = test_dict['X'][:4], test_dict['y'][:4]
predict_gru = conv_gru_model.predict(X_test)[-1]

X_test, y_test = test_dict['X'][:4], test_dict['y'][:4]
predict = conv_model.predict(X_test)[-1]

In [80]:
i = np.random.randint(low=0, high=predict.shape[0])
frame = np.random.randint(low=0, high=predict.shape[1])
print(i, frame)

predict_lbl_gru = label((predict_gru[:,frame,:,:,1]>0.5).astype('int'))
predict_lbl = label((predict[:,frame,:,:,1]>0.5).astype('int'))

0 29


### Featurenet (no GRU)

In [81]:
%%time
m = metrics.Metrics('deepcell',seg=True)

y_true_lbl = y_test[:,frame,:,:,0].astype('int')
y_pred_lbl = predict_lbl

m.calc_object_stats(y_true_lbl,y_pred_lbl)

INFO:tensorflow:0 samples processed

____________Object-based statistics____________

Number of true cells:		 37
Number of predicted cells:	 33

True positives:  11	Accuracy:   29.73%

False positives: 22	Perc Error: 45.83%
False negatives: 26	Perc Error: 54.169999999999995%
Merges:		 0	Perc Error: 0.0%
Splits:		 0	Perc Error: 0.0%

SEG: nan 

CPU times: user 135 ms, sys: 0 ns, total: 135 ms
Wall time: 133 ms


### Featurenet with GRU

In [82]:
%%time
m = metrics.Metrics('deepcell',seg=True)

y_true_lbl = y_test[:,frame,:,:,0].astype('int')
y_pred_lbl = predict_lbl_gru

m.calc_object_stats(y_true_lbl,y_pred_lbl)

INFO:tensorflow:0 samples processed

____________Object-based statistics____________

Number of true cells:		 37
Number of predicted cells:	 51

True positives:  25	Accuracy:   67.57%

False positives: 22	Perc Error: 73.33%
False negatives: 4	Perc Error: 13.33%
Merges:		 4	Perc Error: 13.33%
Splits:		 0	Perc Error: 0.0%

SEG: nan 

CPU times: user 161 ms, sys: 4.4 ms, total: 165 ms
Wall time: 162 ms
